In [42]:
# Opzionale, scarica le sprite dei Pokémon per il gioco
!python ./scripts/sprite_downloader.py

Starting sprite download...
Downloaded: Charmeleon_S.gif
Downloaded: Charmeleon_F.gif
Downloaded: Ivysaur_F.gif
Downloaded: Bulbasaur_S.gif
Downloaded: Bulbasaur_F.gif
Downloaded: Squirtle_F.gif
Downloaded: Wartortle_S.gif
Downloaded: Charmander_S.gif
Downloaded: Squirtle_S.gif
Downloaded: Wartortle_F.gif
Downloaded: Venusaur_F.gif
Downloaded: Venusaur_S.gif
Downloaded: Charizard_F.gif
Downloaded: Blastoise_F.gif
Downloaded: Caterpie_S.gif
Downloaded: Metapod_S.gif
Downloaded: Metapod_F.gif
Downloaded: Charizard_S.gif
Downloaded: Charmander_F.gif
Downloaded: Mega Charizard X_S.gif
Downloaded: Caterpie_F.gif
Downloaded: Mega Venusaur_F.gif
Downloaded: Butterfree_F.gif
Downloaded: Mega Venusaur_S.gif
Downloaded: Mega Charizard X_F.gif
Downloaded: Blastoise_S.gif
Downloaded: Mega Charizard Y_S.gif
Downloaded: Weedle_F.gif
Downloaded: Butterfree_S.gif
Downloaded: Mega Blastoise_S.gif
Downloaded: Weedle_S.gif
Downloaded: Mega Charizard Y_F.gif
Downloaded: Kakuna_S.gif
Downloaded: Kakuna_F.g

In [1]:
# Librerie base
import pandas as pd
import numpy as np

# Machine Learning
import joblib
from sklearn.preprocessing import FunctionTransformer

# Management di file
import os

# Flask
from flask import Flask, request, jsonify, render_template

In [2]:
# Definizione dei percorsi e dei parametri globali
DATASET_BASE_PATH: str = './datasets'
COMBATS_PATH: str = f"{DATASET_BASE_PATH}/combats.csv"
TYPE_CHART_PATH: str = f"{DATASET_BASE_PATH}/type_chart.csv"
POKEMON_PATH: str = f"{DATASET_BASE_PATH}/pokemon.csv"
MODEL_DIR: str = './models'

In [3]:
def load_pokemon_data(combats_path: str, pokemon_path: str) -> pd.DataFrame:
   df_combats = pd.read_csv(combats_path)
   df_pokemon = pd.read_csv(pokemon_path, index_col="#").fillna("None")
   # Prerprocessing del campo winner per avere 0 se vince il primo Pokemon e 1 se vince il_So
   df_combats["Winner"] = (df_combats["Winner"] != df_combats["First_pokemon"]).astype(int)
   # Merge first and second Pokémon stats
   return df_combats.merge(
      df_pokemon.add_suffix("_F"), how="left",
      left_on="First_pokemon", right_index=True
   ).merge(
      df_pokemon.add_suffix("_S"), how="left",
      left_on="Second_pokemon", right_index=True
   ).drop(
      columns=["First_pokemon", "Second_pokemon"]
   )

In [4]:
# Caricamento dei DataFrame
df_type_chart = pd.read_csv(TYPE_CHART_PATH).fillna("None")
df_pokemon = pd.read_csv(POKEMON_PATH, index_col="#").fillna("None")
df_combats: pd.DataFrame = load_pokemon_data(COMBATS_PATH, POKEMON_PATH)

In [5]:
from utils import function_transformer
data_transformer: FunctionTransformer = FunctionTransformer(func=function_transformer, kw_args={'dataframe_type_chart': df_type_chart}).set_output(transform="pandas")

In [ ]:
app: Flask = Flask(__name__, template_folder='webpage')

models = os.listdir(MODEL_DIR)
model = joblib.load(os.path.join(MODEL_DIR, models[0]))

@app.route('/', methods=['GET'])
def one_v_one_battle():
    return render_template('1v1_battle.html', models=models, pokemon_choices=df_pokemon['Name'])

@app.route('/change_model', methods=['POST'])
def change_model():
    data = request.get_json()
    if not data:
        return jsonify({"error": "Invalid JSON."}), 400
    model_filename: str = data.get('model_file')
    if not model_filename or model_filename not in models:
        return jsonify({"error": "Model file not found."}), 404
    global model
    model = joblib.load(os.path.join(MODEL_DIR, model_filename))
    return jsonify({"message": f"Model changed to {model_filename}."}), 200

@app.route('/precdict_battle', methods=['POST'])
def predict_battle():
    data = request.get_json()
    if not data:
        return jsonify({"error": "Invalid JSON."}), 400
    p1: str = data.get("pokemon_first")
    p2: str = data.get("pokemon_second")
    if not p1 or not p2:
        return jsonify({"error": "Both Pokémon must be specified."}), 400
    first_data = df_pokemon.loc[df_pokemon['Name'] == p1].add_suffix(suffix='_F')
    second_data = df_pokemon.loc[df_pokemon['Name'] == p2].add_suffix(suffix='_S')
    winner_id = model.predict(first_data.merge(second_data, how='cross'))
    winner = p1 if winner_id == 0 else p2
    confidence = np.max(model.predict_proba(first_data.merge(second_data, how='cross')), axis=1)[0] * 100
    return jsonify({
        "winner": winner,
        "confidence": float(confidence)
    })

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2025 10:17:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:41] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:42] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:43] "POST /change_model HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:43] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:45] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:46] "POST /change_model HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:46] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:47] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:48] "POST /change_model HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:49] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:50] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 10:17:51] "POST /change_model